In [1]:
import pandas as pd

df = pd.read_csv('./datasets/projectbenyehuda/benyehuda_nikud_dataset.csv')

In [2]:
from transformers import AutoTokenizer

# Load tokenizer once
TOKENIZER_NAME = "dicta-il/dictabert-large-char-menaked"
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

In [3]:
# Display the first few rows of the dataframe
print(df.head())

# Show basic info about the dataframe
print(df.info())

# Show summary statistics for the length of text and article_length columns
df['text_length'] = df['text'].apply(len)
print(df[['text_length']].describe())

# Check for missing values
print(df.isnull().sum())

# Display the distribution of nikud_mask lengths
df['nikud_mask_length'] = df['nikud_mask'].apply(lambda x: len(x.split()))
print(df['nikud_mask_length'].value_counts().head())

      id           title                                               text  \
0  10026  איש הולך ברחוב  איש הולך ברחוב מאת רחל נגב אִישׁ הֹולֵךְ בָּרְ...   
1  10026  איש הולך ברחוב  אוּלַי אֵין לוֹ מִישֶׁהוּ לְדַבֵּר עִמּוֹ בַּב...   
2  10026  איש הולך ברחוב               וַדַּאי אֵין מִי שֶׁיְּדַבֵּר עִמּוֹ   
3  10026  איש הולך ברחוב                 אִישׁ הֹולֵךְ בָּרְחוֹב וּמְדַבֵּר   
4  10026  איש הולך ברחוב  אֶפְשָׁר לוֹמַר: אִישׁ הֹולֵךְ בָּרְחוֹב וּמְד...   

                                          nikud_mask  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, ...  
2  [0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, ...  
3  [0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, ...  
4  [0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, ...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114261 entries, 0 to 114260
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0 

In [4]:
import re

NIKUD_PATTERN = re.compile(
    '['
    '\u05B0'  # sheva
    '\u05B1'  # hataf segol
    '\u05B2'  # hataf patah
    '\u05B3'  # hataf qamats
    '\u05B4'  # hiriq
    '\u05B5'  # tsere
    '\u05B6'  # segol
    '\u05B7'  # patah
    '\u05B8'  # qamats
    '\u05B9'  # holam
    '\u05BB'  # qubuts
    '\u05BC'  # dagesh or mapiq
    '\u05BD'  # meteg
    '\u05BF'  # rafe (rare)
    '\u05C1'  # shin dot
    '\u05C2'  # sin dot
    '\u05C7'  # qamats qatan
    ']'
)

In [5]:
# Check if each row contains at least one word with nikud using NIKUD_PATTERN
def has_word_with_nikud(text):
    return any(NIKUD_PATTERN.search(word) for word in text.split())

# Apply the function to the 'text' column and check if all rows satisfy the condition
all_have_nikud = df['text'].apply(has_word_with_nikud).all()
print("All rows contain at least one word with nikud:", all_have_nikud)

All rows contain at least one word with nikud: True


In [ ]:
import ast

def find_next_space(tokens, index):
    """Find the next space after the given index in the token list."""
    for i in range(index, len(tokens)):
        if tokens[i] == " ":
            return i
    return None

def find_previous_space(tokens, index):
    """Find the previous space before the given index in the token list."""
    for i in range(index - 1, -1, -1):
        if tokens[i] == " ":
            return i
    return None

# Find rows where the nikud_mask contains only one to three 1s in a row (consecutive)
def has_1s_run(text, mask, min_run=1, max_run=3):
    mask_str = ''.join(str(x) for x in mask)
    tokens = tokenizer.tokenize(text)
    # Check for runs of 1s of length 1, 2, or 3
    for run_length in range(max_run, min_run - 1, -1):
        i = mask_str.find(f"{'1' * run_length}")
        if i != -1:
            prev = find_previous_space(tokens, i)
            prevprev = find_previous_space(tokens, prev) if prev is not None else None
            prevvalid = True
            if prevprev is not None:
                prevvalid = '1' not in mask_str[prevprev+1:prev]
            elif prev is not None:
                prevvalid = '1' not in mask_str[0:prev]
            next = find_next_space(tokens, i + run_length)
            nextnext = find_next_space(tokens, next + 1) if next is not None else None
            nextvalid = True
            if nextnext is not None:
                nextvalid = '1' not in mask_str[next+1:nextnext]
            elif next is not None:
                nextvalid = '1' not in mask_str[next+1:]

            if prevvalid and nextvalid:
                return True
    return False

def column_filter(row):
    text = row['text']
    mask = ast.literal_eval(row['nikud_mask'])
    return has_1s_run(text, mask, min_run=1, max_run=3)

In [7]:
filtered_rows = df.sample(5)
filtered_rows.iloc[0]['nikud_mask']


'[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'

In [8]:
filtered_rows = filtered_rows[filtered_rows.apply(column_filter, axis=1)]
filtered_rows

,id,title,text,nikud_mask,text_length,nikud_mask_length
91303,50048,אבות ובנים,"אתה נשמה עדינה, רכרוכי, כיצד תוכל לשנוא!… אתה ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",115,119
20603,14166,אורליה ספר החזיונות והתפילות,"והנה עלומי ניצב בשולי הדרך, ולבי נרגע מגלוּתו","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",45,46
61187,33859,בלילות,כרמיבַּלֵּילוֹת מאת ט,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...",21,18
34039,22580,אילן מאליונר,"אז יאללה, תיהנה מהחיים אילן כל זמן שאתה צעיר, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",148,149
35505,23942,ספר הגעגועים רומאן,"ואכן, צבי ואני עשינו כמיטב יכולתנו במשׂור ובפצ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",90,91


In [9]:
filtered_rows['text'].values[0]

'אתה נשמה עדינה, רכרוכי, כיצד תוכל לשנוא!… אתה מפחד, סומך אך מעט על עצמך…” “ואתה,” הפסיק אותו אַרקָדי, “סומך על עצמך'

In [10]:
from tqdm.notebook import tqdm

tqdm.pandas()  # Enable tqdm for pandas

# filtered_df = df[df['nikud_mask'].apply(lambda x: has_1s_run(ast.literal_eval(x)) if isinstance(x, str) else has_1s_run(x))]
filtered_df = df[df.progress_apply(column_filter, axis=1)]
filtered_df.shape

  0%|          | 0/114261 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2492 > 2048). Running this sequence through the model will result in indexing errors


(60315, 6)

In [11]:
sample_text = filtered_df.head(1).values[0][2]
sample_mask = filtered_df.head(1).values[0][3]
# has_1s_run(sample_text, ast.literal_eval(sample_mask))


In [12]:
filtered_df.head(5)

,id,title,text,nikud_mask,text_length,nikud_mask_length
6,10290,הנפט זורם לים התיכון,הנפט זורם לים התיכון מאת ישראל זרחי חלק ראשון ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",183,180
7,10290,הנפט זורם לים התיכון,כלים יימָצאו כבר,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",16,17
8,10290,הנפט זורם לים התיכון,לא היה דמיון בספָרות המוזרות ולא טעם בשאר הלימ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",50,51
9,10290,הנפט זורם לים התיכון,אותו לא ירמו – אבל כשעבר השיעור מבלי שהמורה יע...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",388,389
10,10290,הנפט זורם לים התיכון,"בלילה, משחזר אבא בעגלתו מהשדה ונכנס לאורווה במ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",228,229


In [13]:
filtered_df.to_csv('./datasets/projectbenyehuda/benyehuda_nikud_dataset_filtered.csv', index=False)